<h1>Lois de probabilités sur un domaine infini</h1>

Les lois étudiées ci-dessous ont pour support $\mathbb{N}$.

<h2> Loi de Bernoulli </h2>

La loi de Bernoulli décrit un tirage aléatoire à deux résultats possibles (succès et échec, numérotés 1 et 0), de probabilités respectives $p$ et $1-p$.

$P (X=x)=\left\{\begin{array}{ll}
p&\quad {\mbox{si }}x=1,\\
1-p&\quad {\mbox{si }}x=0,\\
0&\quad {\mbox{sinon.}}
\end{array}\right.$

À l'aide de la fonction <b>Random.float</b>, écrire une fonction $Bernoulli(p)$ qui suit la loi indiquée.

In [1]:
#use "topfind";;
#require "plplot";;
open Plplot;;
module P = Plot;;
let couleurs_list = [[ 0;255;255;255]; (*`white*)
                     [ 1;  0;  0;  0]; (*`black*)
                     [ 2;  0;  0;255]; (*`blue*)
                     [ 3;255;  0;  0]; (*`red*)
                     [ 4;165; 42; 42]; (*`brown*)
                     [ 5;  0;  0;  0]; [ 6;  0;  0;  0]; [ 7;  0;  0;  0]; [ 8;  0;  0;  0]; [ 9;  0;  0;  0]; 
                     [10;200;200;200]; (*`gray*)
                     [11;  0;255;255]; (*`light_blue*)
                     [12;  0;255;  0]; (*`green*)
                     [13;255;255;  0]; (*`yellow*)
                     [14;255;  0;255]; (*`pink*)
                     [15;160;  0;213]; (*`purple*) ]
let rec loop couleurs_list = match couleurs_list with
    | [n;r;g;b]::tl -> plscol0 n r g b; loop tl
    | _ -> ();;
let couleurs = (fun () -> plscolbg 255 255 255; loop couleurs_list)
let initialisation filename xmin xmax ymin ymax = 
        P.init (xmin, ymin) (xmax, ymax) `greedy (`svg `core) ~filename:(filename^".svg") ~pre:couleurs
let xlabel texte = P.text_outside `black (`bottom 0.5) 3. texte
let ylabel texte = P.text_outside `black (`left 0.5) 5. texte 

- : unit = ()
Findlib has been successfully loaded. Additional directives:
  #require "package";;      to load a package
  #list;;                   to list the available packages
  #camlp4o;;                to load camlp4 (standard syntax)
  #camlp4r;;                to load camlp4 (revised syntax)
  #predicates "p,q,...";;   to set these predicates
  Topfind.reset();;         to force that packages will be reloaded
  #thread;;                 to enable threads

- : unit = ()


/usr/lib/ocaml/plplot: added to search path
/usr/lib/ocaml/plplot/plplot.cma: loaded


module P = Plplot.Plot

val couleurs_list : int list list =
 [[0; 255; 255; 255]; [1; 0; 0; 0]; [2; 0; 0; 255]; [3; 255; 0; 0];
 [4; 165; 42; 42]; [5; 0; 0; 0]; [6; 0; 0; 0]; [7; 0; 0; 0]; [8; 0; 0; 0];
 [9; 0; 0; 0]; [10; 200; 200; 200]; [11; 0; 255; 255]; [12; 0; 255; 0];
 [13; 255; 255; 0]; [14; 255; 0; 255]; [15; 160; 0; 213]]

val loop : int list list -> unit = <fun>

val couleurs : unit -> unit = <fun>

val initialisation : string -> float -> float -> float -> float -> P.stream_t =
 <fun>

val xlabel : string -> P.plot_t = <fun>

val ylabel : string -> P.plot_t = <fun>

In [2]:
let bar ?x_width ?x_offset color xs ys = 
    let x_width_ = match x_width with
        |None -> 0.4
        |Some x -> x in
    let x_offset_ = match x_offset with
        |None -> 0.
        |Some x -> x in
    
    let x = Array.make (4*(Array.length xs)) 0. in
    let y = Array.make (4*(Array.length xs)) 0. in
    for i = 0 to Array.length xs - 1 do
        x.(4*i)   <- x_offset_ +. xs.(i) -. x_width_;
        x.(4*i+1) <- x_offset_ +. xs.(i) -. x_width_;
        x.(4*i+2) <- x_offset_ +. xs.(i) +. x_width_;
        x.(4*i+3) <- x_offset_ +. xs.(i) +. x_width_;
        y.(4*i)   <- 0.;
        y.(4*i+1) <- ys.(i);
        y.(4*i+2) <- ys.(i);
        y.(4*i+3) <- 0.;
    done; P.polygon ~fill:true color x y;;

val bar :
 ?x_width:float ->
 ?x_offset:float -> P.color_t -> float array -> float array -> P.plot_t =
 <fun>

In [3]:
let range start stop step =
   let rec range i acc =
      if step>0 && i>=stop then List.rev acc else
      if step<0 && i<=stop then List.rev acc else
      range (i+step) (i::acc) in
   range start [];;

val range : int -> int -> int -> int list = <fun>

In [4]:
open Random;;
Random.self_init;;
let bernoulli p = if  Random.float 1. <= p then 1 else 0;;

- : unit -> unit = <fun>

val bernoulli : float -> int = <fun>

<h2> Loi géométrique </h2>

La loi géométrique décrit le nombre d'essais nécessaires, dans une suite de tirages Bernoulli, avant d'obtenir un succès.

<h3> Définition 1</h3>

La probabilité $P(X=k)$ correspond à la probabilité d'obtenir dans une succession de $k$ épreuves de Bernoulli, $k-1$ échecs suivis d'un succès.

Ecrire une fonction $f\_{geometrique}(x,p)$ en simulant des tirages aléatoires.

On fera des moyennes sur 10 000 essais.

In [5]:
let f_geometrique x p =
    let nbre_essais = 100000 in
    let rec loop1 num_essai res =
        if num_essai < nbre_essais then
        begin
        let rec loop2 valeur =
            if bernoulli p == 1 && valeur <= x 
            then loop2 (valeur+1)
            else valeur-1 (*x-1 echecs + 1 succes *) in
        if loop2 0 = x 
        then loop1 (num_essai+1) (res+1)
        else loop1 (num_essai+1) res;
        end
        else
        float_of_int res /. (float_of_int nbre_essais) in
        
    loop1 0 0;;

val f_geometrique : int -> float -> float = <fun>

Tracer la loi géométrique $f\_{geometrique}(x,0.5)$ sur l'intervalle $[[0,10]]$ 

In [6]:
let liste_x = range 0 11 1 in
let xs = Array.of_list (List.map float_of_int liste_x) in
let ys = Array.of_list (List.map (fun x -> f_geometrique x 0.5) liste_x) in
let p = initialisation "graph1" (-.2.) 12. 0. 0.5 in
P.plot ~stream:p [bar `blue xs ys];
P.finish ~stream:p ();;

- : unit = ()

<img src="graph1.svg" width=500/>

<h3> Définition 2</h3>

La probabilité $P(X=k)$ correspond à $P(X=k)=(1-p)^{k-1}p$.

Redéfinir la fonction $f\_{geometrique}(x,p)$

In [7]:
let f_geometrique x p =
    if x>0 then (1.-.p)**(float_of_int x-.1.)*.p else 0.

val f_geometrique : int -> float -> float = <fun>

Tracer la loi géométrique $f\_{geometrique}(x,0.5)$ sur l'intervalle $[[0,10]]$ 

In [8]:
let liste_x = range 0 11 1 in
let xs = Array.of_list (List.map float_of_int liste_x) in
let ys = Array.of_list (List.map (fun x -> f_geometrique x 0.5) liste_x) in
let p = initialisation "graph2" (-.2.) 12. 0. 0.5 in
P.plot ~stream:p [bar `blue xs ys];
P.finish ~stream:p ();;

- : unit = ()

<img src="graph2.svg" width=500/>

Tracer la fonction de répartition de la loi géométrique $f\_{geometrique}(x,0.5)$ sur l'intervalle $[[0,10]]$

In [9]:
let repartition_geometrique x p =
    let rec loop var res =
        if var <= x then loop (var+1) (res+.(f_geometrique var p)) else res in
    loop 0 0.;;

let liste_x = range 0 11 1 in
let xs = Array.of_list (List.map float_of_int liste_x) in
let ys = Array.of_list (List.map (fun x -> repartition_geometrique x 0.5) liste_x) in
let p = initialisation "graph3" (-.2.) 12. 0. 1.0 in
P.plot ~stream:p [bar `blue xs ys];
P.finish ~stream:p ();;

val repartition_geometrique : int -> float -> float = <fun>

- : unit = ()

<img src="graph3.svg" width=500/>

<h2> Loi de binomiale négative </h2>

<h3> Définition 1</h3>

La loi <b>binomiale négative</b> est une distribution de probabilité discrète. Elle décrit la situation suivante : une expérience consiste en une série de tirages indépendants, donnant un "succès" avec probabilité $p$ (constante durant toute l'expérience) et un "échec" avec une probabilité complémentaire. Cette expérience se poursuit jusqu'à l'obtention d'un nombre donné $n$ de succès. La variable aléatoire représentant le nombre d'échecs (avant l'obtention du nombre donné $n$ de succès) suit alors une loi binomiale négative. Ses paramètres sont $n$, le nombre de succès attendus, et $p$, la probabilité d'un succès.


Ecrire une fonction $f\_{Pascal}(x,n,p)$ en simulant des tirages aléatoires.

On fera des moyennes sur 10 000 essais.

In [10]:
let f_Pascal x n p =
    let nbre_essais = 100000 in
    let rec loop1 num_essai res =
        if num_essai < nbre_essais then begin
        let rec loop2 succes echec =
            if succes != n then 
               if bernoulli p = 0
               then loop2 (succes+1) echec
               else loop2 succes (echec+1)
            else echec in
        if loop2 0 0 = x
        then loop1 (num_essai+1) (res+1)
        else loop1 (num_essai+1) res
        end else
        float_of_int res /. (float_of_int nbre_essais) in
    loop1 0 0;;

val f_Pascal : int -> int -> float -> float = <fun>

Tracer la loi $f\_{Pascal}(x,3,.5)$ sur l'intervalle $[[0,10]]$

In [11]:
let liste_x = range 0 11 1 in
let xs = Array.of_list (List.map float_of_int liste_x) in
let ys = Array.of_list (List.map (fun x -> f_Pascal x 3 0.5) liste_x) in
let p = initialisation "graph4" (-.2.) 12. 0. 0.2 in
P.plot ~stream:p [bar `blue xs ys];
P.finish ~stream:p ();;

- : unit = ()

<img src="graph4.svg" width=500/>

<h3> Définition 2</h3>

La loi binomiale négative, de paramètres $n$ et $p$, est la loi de probabilité discrète d'une variable aléatoire X dont la fonction de masse est donnée par :

$P(X=k;n,p)={k+n-1 \choose k}\cdot p^{n}\cdot q^{k}$  pour k = 0, 1, 2,$\dots$

À l'aide d'une fonction <b>factorial</b>, écrire une fonction $f\_{Pascal}(x,p,n)$ qui suit la loi indiquée.

In [12]:
#require "num";;
open Num;;
let rec factorial n = 
    if n=0 then Int 1 else Int n */ (factorial (n-1));;
    
let choose k n =
    (* on évite de calculer N! qui peut être très grand
       alors qu'on a simplement besoin de N!/(N-k)! *)
    let rec loop valeur res =
        if valeur<n+1 then loop (valeur+1) (res*/ (Int valeur))
        else res // (factorial k) in
    loop (n-k+1) (Int 1);;

let f_Pascal x n p =
    if x>=0 && n>0
    then float_of_num (choose x (x+n-1)) *. p**(float_of_int n) *. (1.-.p)**(float_of_int x)
    else 0.;;

/usr/lib/ocaml/nums.cma: loaded
/usr/lib/ocaml/num-top: added to search path
/usr/lib/ocaml/num-top/num_top.cma: loaded


val factorial : int -> Num.num = <fun>

val choose : int -> int -> Num.num = <fun>

val f_Pascal : int -> int -> float -> float = <fun>

Tracer la loi $f\_{Pascal}(x,3,.5)$ sur l'intervalle $[[0,10]]$

In [13]:
let liste_x = range 0 11 1 in
let xs = Array.of_list (List.map float_of_int liste_x) in
let ys = Array.of_list (List.map (fun x -> f_Pascal x 3 0.5) liste_x) in
let p = initialisation "graph5" (-.2.) 12. 0. 0.2 in
P.plot ~stream:p [bar `blue xs ys];
P.finish ~stream:p ();;

- : unit = ()

<img src="graph5.svg" width=500/>

<h2> Loi de Poisson </h2>

La loi de Poisson décrit la probabilité d'observer un certain nombre d'événements aléatoires dans un intervalle continu (durée, longueur).

Il faut que ces évènements se produisent avec une fréquence moyenne connue et indépendamment du temps écoulé depuis l'évènement précédent.


<h3> Définition 1</h3>

On rappelle que la loi binomiale, de paramètres $n$ et $p$, est la loi de probabilité d'une variable aléatoire $X$ égale au nombre de succès rencontrés au cours d'une répétition de $n$ épreuves de Bernoulli, $p$ étant la probabilité de succès dans chacune d'entre elles.

$\newcommand{grtsim}{\geq}$

La loi de <b>Poisson</b> correspond à la loi binomiale lorsque $n\to\infty$ avec $n\times p=\lambda$. En pratique, il suffit que $n \grtsim 100$.

Ecrire une fonction $f\_{Poisson}(x,lambda)$ en simulant des tirages aléatoires.

On fera des moyennes sur 10 000 essais.

In [14]:
let binomiale x p n =
    let nbre_essais = 10000 in
    let rec loop1 num_essai res =
        if num_essai < nbre_essais then begin
        let rec loop2 num valeur =
            if num<n 
            then loop2 (num+1) (valeur + (bernoulli p))
            else valeur in
        if loop2 0 0 = x
        then loop1 (num_essai+1) (res+1)
        else loop1 (num_essai+1) res;
        end else
        float_of_int res /. (float_of_int nbre_essais) in
    loop1 0 0;;

let f_Poisson x lambda =
    binomiale x (lambda/.100.) 100;;

val binomiale : int -> float -> int -> float = <fun>

val f_Poisson : int -> float -> float = <fun>

Tracer, sur un même graphique, les diagrammes pour $p=0.2$, $p=0.5$ et $p=0.8$ sur l'intervalle $[[0,5]]$

In [15]:
let liste_x = range 0 6 1 in
let array_lambda = [|0.2; 0.5; 0.8|] in
let legende = Array.init 3 (fun i-> "lambda="^(string_of_float array_lambda.(i))) in
                                    
let xs = Array.of_list (List.map float_of_int liste_x) in
let ys = Array.of_list (List.map (fun x -> f_Poisson x array_lambda.(0)) liste_x) in
let ys' =  Array.of_list (List.map (fun x -> f_Poisson x array_lambda.(1)) liste_x) in
let ys'' =  Array.of_list (List.map (fun x -> f_Poisson x array_lambda.(2)) liste_x) in

let p = initialisation "graph6" (-.1.) 6. 0. 0.9 in
P.plot ~stream:p [bar ~x_width:0.1 ~x_offset:(-.0.2) `blue xs ys;
                  bar ~x_width:0.1 ~x_offset:0. `green xs ys';
                  bar ~x_width:0.1 ~x_offset:0.2 `red xs ys'';
                  P.legend [[P.line_legend legende.(0) `blue];
                            [P.line_legend legende.(1) `green];
                            [P.line_legend legende.(2) `red]]];
P.finish ~stream:p ();;

- : unit = ()

<img src="graph6.svg" width=500/>

<h3> Définition 2</h3>

Si le nombre moyen d'occurrences dans cet intervalle est $\lambda$, alors la probabilité qu'il existe exactement $k$ occurrences ($k$ étant un entier naturel, $k = 0, 1, 2,\dots$) est

$P(X = k)= \frac{\lambda ^k}{k!}e^{-\lambda}$

On dit alors que $X$ suit la loi de Poisson de paramètre $\lambda$.

À l'aide de fonctions <b>exp</b> et <b>factorial</b>, écrire une fonction $f\_{Poisson}(x,lambda)$ qui suit la loi indiquée.

In [16]:
let f_Poisson x lambda = lambda**(float_of_int x)*.exp(-.lambda)/.(float_of_num (factorial x))

val f_Poisson : int -> float -> float = <fun>

Tracer, sur un même graphique, les diagrammes pour $p=0.2$, $p=0.5$ et $p=0.8$ sur l'intervalle $[[0,5]]$

In [17]:
let liste_x = range 0 6 1 in
let array_lambda = [|0.2; 0.5; 0.8|] in
let legende = Array.init 3 (fun i-> "lambda="^(string_of_float array_lambda.(i))) in
                                    
let xs = Array.of_list (List.map float_of_int liste_x) in
let ys = Array.of_list (List.map (fun x -> f_Poisson x array_lambda.(0)) liste_x) in
let ys' =  Array.of_list (List.map (fun x -> f_Poisson x array_lambda.(1)) liste_x) in
let ys'' =  Array.of_list (List.map (fun x -> f_Poisson x array_lambda.(2)) liste_x) in

let p = initialisation "graph7" (-.1.) 6. 0. 0.9 in
P.plot ~stream:p [bar ~x_width:0.1 ~x_offset:(-.0.2) `blue xs ys;
                  bar ~x_width:0.1 ~x_offset:0. `green xs ys';
                  bar ~x_width:0.1 ~x_offset:0.2 `red xs ys'';
                  P.legend [[P.line_legend legende.(0) `blue];
                            [P.line_legend legende.(1) `green];
                            [P.line_legend legende.(2) `red]]];
P.finish ~stream:p ();;

- : unit = ()

<img src="graph7.svg" width=500/>